In [ ]:
! python -m pip install gurobipy
! python -m pip install pandas

In [ ]:
import json
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# Lettura file CSV
df = pd.read_csv("data/dieta2.csv")
"""
imposto la prima colonna ('i' in questo caso) come indice di riga per poter accedere ad una riga del file utilizzando .loc
riga di codice equivalente per ottenere lo stesso risultato: df = df.set_index('i')
"""
idx = df.columns.tolist()[0]
df = df.set_index(f'{idx}')

# Definizione degli insiemi
"""
Ottengo l’insieme I (cibi disponibili):
lista dei nomi delle colonne del dataframe, esclusa la colonna 'domanda'
I = ['verdura', 'carne', 'pesce', ...]
"""
I = df.drop(columns="domanda").columns.tolist()

"""
Ottengo l’insieme J (nutrienti):
lista dei nomi delle righe del dataframe, esclusa la riga 'costo'
J = ['proteine', 'ferro', 'calcio', ...]
"""
J = df.drop(index="costo").index.tolist()

# Definizione dei parametri
"""
Seleziono la riga con indice 'costo' e la converto in un dizionario:
- chiavi = cibi (colonne del DataFrame)
- valori = costo unitario di ciascun cibo

C = { 'verdura': 2.0,
      'carne': 1.5,
      ... }
"""
C = df.loc["costo"].to_dict()


"""
Seleziono la colonna 'domanda' e la converto in un dizionario:
- chiavi = nutrienti (righe del DataFrame)
- valori = quantità minima richiesta per ciascun nutriente

D = { 'proteine': 50,
      'ferro': 20,
      ... }
"""
D = df["domanda"].to_dict()

"""
Creo la matrice A eliminando le righe e le colonne 'costo' e 'domanda'
- la riga 'costo' è eliminata perché non è un nutriente
- la colonna 'domanda' è eliminata perché non è un cibo

A[j,i] indica quanta quantità del nutriente j fornisce una unità del cibo i.
NB: le righe = nutrienti, le colonne = cibi.
"""
A  = df.drop(index="costo").drop(columns="domanda")


print("Costi (C):", C)
print("Domande (D):", D)
print("Matrice (A):\n", A)
print("Cibi (I):", I)
print("Nutrienti (J):", J)


# Creazione modello
m = gp.Model("dieta2")
m.setParam("OutputFlag", 0)

# Creazione variabili decisionali
x = m.addVars(I, vtype=GRB.INTEGER, lb=0, name="x")

# Creazione vincoli
for j in J:
    sum = gp.quicksum(A.loc[j, i] * x[i] for i in I)
    m.addConstr(
        sum >= D[j],
        name=f"nutriente_{j}",
    )

# Nuovo vincolo: almeno 3kg di alimenti a base di pesce azzurro
m.addConstr(x['pesce'] >= 3)

# Definizione o.f.
m.setObjective(gp.quicksum(C[i] * x[i] for i in I), GRB.MINIMIZE)

# Ottimizzazione
m.optimize()

# Visualizzazione risultati
if m.status == GRB.OPTIMAL:
    print(f"O.f. (costo) = {m.objVal:.3f}\n")
    for i in I:
        print(f"x[{i}] = {int(x[i].X):.2f}")